# 構造化出力評価クックブック

このノートブックでは、OpenAI **Evals**フレームワークを使用して**大規模言語モデルに構造化出力を生成させるタスクをテスト、評価、改善する方法**について、焦点を絞った実行可能な例を通して説明します。

> **なぜこれが重要なのか？**  
> 本番システムは多くの場合、JSON、SQL、またはドメイン固有のフォーマットに依存しています。スポットチェックやその場しのぎのプロンプト調整に頼っていると、すぐに破綻してしまいます。代わりに、期待値を自動化されたevalsとして*体系化*し、チームが砂ではなく安全なレンガで構築できるようにすることができます。

## クイックツアー

* **セクション1 – 前提条件**: 環境変数とパッケージのセットアップ  
* **セクション2 – ウォークスルー: コードシンボル抽出**: ソースコードから関数名とクラス名を抽出するモデルの能力を評価するエンドツーエンドのデモ。元のロジックをそのまま保持し、単純にドキュメントを重ねます。  
* **セクション3 – 追加レシピ**: 評価用の追加コードサンプルとして、感情抽出などの一般的な本番環境パターンのスケッチ。
* **セクション4 – 結果の探索**: 実行結果の取得と失敗の詳細調査のための軽量ヘルパー。

## 前提条件

1. **依存関係をインストール**（最小バージョンを表示）：

```bash
pip install --upgrade openai
```

2. **認証**：キーをエクスポートして設定：

```bash
export OPENAI_API_KEY="sk‑..."
```

3. **オプション**：評価を一括実行する予定がある場合は、適切な制限を設定した[組織レベルキー](https://platform.openai.com/account/org-settings)をセットアップしてください。

### ユースケース1: コードシンボル抽出

目標は**OpenAI SDK内のPythonファイルから、すべての関数、クラス、定数のシンボルを抽出する**ことです。  
各ファイルに対して、モデルに以下のような構造化されたJSONを出力するよう求めます：

```json
{
  "symbols": [
    {"name": "OpenAI", "kind": "class"},
    {"name": "Evals", "kind": "module"},
    ...
  ]
}
```

ルーブリックモデルが**完全性**（すべてのシンボルを捉えたか？）と**品質**（種類は正しいか？）を1‑7のスケールで評価します。

### カスタムデータセットを用いたコード品質抽出の評価

OpenAI **Evals**フレームワークとカスタムのインメモリデータセットを使用して、コードからシンボルを抽出するモデルの能力を評価する例を見ていきましょう。

### SDKクライアントの初期化
上記でエクスポートした`OPENAI_API_KEY`を使用して`openai.OpenAI`クライアントを作成します。これがないと何も実行されません。

In [11]:
%pip install --upgrade openai pandas rich --quiet



import os
import time
import openai
from rich import print
import pandas as pd

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY") or os.getenv("_OPENAI_API_KEY"),
)


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### データセットファクトリと採点ルーブリック
* `get_dataset`は複数のSDKファイルを読み込んで、小さなインメモリデータセットを構築します。
* `structured_output_grader`は詳細な評価ルーブリックを定義します。
* `client.evals.create(...)`は評価をプラットフォームに登録します。

In [4]:
def get_dataset(limit=None):
    openai_sdk_file_path = os.path.dirname(openai.__file__)

    file_paths = [
        os.path.join(openai_sdk_file_path, "resources", "evals", "evals.py"),
        os.path.join(openai_sdk_file_path, "resources", "responses", "responses.py"),
        os.path.join(openai_sdk_file_path, "resources", "images.py"),
        os.path.join(openai_sdk_file_path, "resources", "embeddings.py"),
        os.path.join(openai_sdk_file_path, "resources", "files.py"),
    ]

    items = []
    for file_path in file_paths:
        items.append({"input": open(file_path, "r").read()})
    if limit:
        return items[:limit]
    return items


structured_output_grader = """
You are a helpful assistant that grades the quality of extracted information from a code file.
You will be given a code file and a list of extracted information.
You should grade the quality of the extracted information.

You should grade the quality on a scale of 1 to 7.
You should apply the following criteria, and calculate your score as follows:
You should first check for completeness on a scale of 1 to 7.
Then you should apply a quality modifier.

The quality modifier is a multiplier from 0 to 1 that you multiply by the completeness score.
If there is 100% coverage for completion and it is all high quality, then you would return 7*1.
If there is 100% coverage for completion but it is all low quality, then you would return 7*0.5.
etc.
"""

structured_output_grader_user_prompt = """
<Code File>
{{item.input}}
</Code File>

<Extracted Information>
{{sample.output_json.symbols}}
</Extracted Information>
"""

logs_eval = client.evals.create(
    name="Code QA Eval",
    data_source_config={
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {"input": {"type": "string"}},
        },
        "include_sample_schema": True,
    },
    testing_criteria=[
        {
            "type": "score_model",
            "name": "General Evaluator",
            "model": "o3",
            "input": [
                {"role": "system", "content": structured_output_grader},
                {"role": "user", "content": structured_output_grader_user_prompt},
            ],
            "range": [1, 7],
            "pass_threshold": 5.5,
        }
    ],
)

### モデル実行の開始
ここでは、同じ評価に対して2つの実行を開始します：1つは**Completions**エンドポイントを呼び出すもの、もう1つは**Responses**エンドポイントを呼び出すものです。

In [5]:
### Kick off model runs
gpt_4one_completions_run = client.evals.runs.create(
    name="gpt-4.1",
    eval_id=logs_eval.id,
    data_source={
        "type": "completions",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset(limit=1)],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {"type": "input_text", "text": "You are a helpful assistant."},
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Extract the symbols from the code file {{item.input}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 10000,
            "top_p": 0.9,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "python_symbols",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "symbols": {
                                "type": "array",
                                "description": "A list of symbols extracted from Python code.",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "name": {"type": "string", "description": "The name of the symbol."},
                                        "symbol_type": {
                                            "type": "string", "description": "The type of the symbol, e.g., variable, function, class.",
                                        },
                                    },
                                    "required": ["name", "symbol_type"],
                                    "additionalProperties": False,
                                },
                            }
                        },
                        "required": ["symbols"],
                        "additionalProperties": False,
                    },
                    "strict": True,
                },
            },
        },
    },
)

gpt_4one_responses_run = client.evals.runs.create(
    name="gpt-4.1-mini",
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset(limit=1)],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {"type": "input_text", "text": "You are a helpful assistant."},
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Extract the symbols from the code file {{item.input}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1-mini",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 10000,
            "top_p": 0.9,
            "text": {
                "format": {
                    "type": "json_schema",
                    "name": "python_symbols",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "symbols": {
                                "type": "array",
                                "description": "A list of symbols extracted from Python code.",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "name": {"type": "string", "description": "The name of the symbol."},
                                        "symbol_type": {
                                            "type": "string",
                                            "description": "The type of the symbol, e.g., variable, function, class.",
                                        },
                                    },
                                    "required": ["name", "symbol_type"],
                                    "additionalProperties": False,
                                },
                            }
                        },
                        "required": ["symbols"],
                        "additionalProperties": False,
                    },
                    "strict": True,
                },
            },
        },
    },
)

### ユーティリティポーラー
次に、すべての実行が完了するまで待機する単純なループを使用し、その後各実行のJSONをディスクに保存して、後で検査したりCI成果物に添付したりできるようにします。

In [7]:
### Utility poller
def poll_runs(eval_id, run_ids):
    while True:
        runs = [client.evals.runs.retrieve(rid, eval_id=eval_id) for rid in run_ids]
        for run in runs:
            print(run.id, run.status, run.result_counts)
        if all(run.status in {"completed", "failed"} for run in runs):
            # dump results to file
            for run in runs:
                with open(f"{run.id}.json", "w") as f:
                    f.write(
                        client.evals.runs.output_items.list(
                            run_id=run.id, eval_id=eval_id
                        ).model_dump_json(indent=4)
                    )
            break
        time.sleep(5)

poll_runs(logs_eval.id, [gpt_4one_completions_run.id, gpt_4one_responses_run.id])

evalrun_68487dcc749081918ec2571e76cc9ef6 completed
ResultCounts(errored=0, failed=1, passed=0, total=1)

evalrun_68487dcdaba0819182db010fe5331f2e completed
ResultCounts(errored=0, failed=1, passed=0, total=1)

### 迅速な検査のための出力の読み込み
両方の実行の出力項目を取得して、印刷や後処理ができるようにします。

In [8]:
completions_output = client.evals.runs.output_items.list(
    run_id=gpt_4one_completions_run.id, eval_id=logs_eval.id
)

responses_output = client.evals.runs.output_items.list(
    run_id=gpt_4one_responses_run.id, eval_id=logs_eval.id
)

### 人間が読みやすいダンプ
補完と応答の並列表示を印刷してみましょう。

In [20]:
from IPython.display import display, HTML

# Collect outputs for both runs
completions_outputs = [item.sample.output[0].content for item in completions_output]
responses_outputs = [item.sample.output[0].content for item in responses_output]

# Create DataFrame for side-by-side display (truncated to 250 chars for readability)
df = pd.DataFrame({
    "Completions Output": [c[:250].replace('\n', ' ') + ('...' if len(c) > 250 else '') for c in completions_outputs],
    "Responses Output": [r[:250].replace('\n', ' ') + ('...' if len(r) > 250 else '') for r in responses_outputs]
})

# Custom color scheme
custom_styles = [
    {'selector': 'th', 'props': [('font-size', '1.1em'), ('background-color', '#323C50'), ('color', '#FFFFFF'), ('border-bottom', '2px solid #1CA7EC')]},
    {'selector': 'td', 'props': [('font-size', '1em'), ('max-width', '650px'), ('background-color', '#F6F8FA'), ('color', '#222'), ('border-bottom', '1px solid #DDD')]},
    {'selector': 'tr:hover td', 'props': [('background-color', '#D1ECF1'), ('color', '#18647E')]},
    {'selector': 'tbody tr:nth-child(even) td', 'props': [('background-color', '#E8F1FB')]},
    {'selector': 'tbody tr:nth-child(odd) td', 'props': [('background-color', '#F6F8FA')]},
    {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('border-radius', '6px'), ('overflow', 'hidden')]},
]

styled = (
    df.style
    .set_properties(**{'white-space': 'pre-wrap', 'word-break': 'break-word', 'padding': '8px'})
    .set_table_styles(custom_styles)
    .hide(axis="index")
)

display(HTML("""
<h4 style="color: #1CA7EC; font-weight: 600; letter-spacing: 1px; text-shadow: 0 1px 2px rgba(0,0,0,0.08), 0 0px 0px #fff;">
Completions vs Responses Output
</h4>
"""))
display(styled)

Completions Output,Responses Output
"{""symbols"":[{""name"":""Evals"",""symbol_type"":""class""},{""name"":""AsyncEvals"",""symbol_type"":""class""},{""name"":""EvalsWithRawResponse"",""symbol_type"":""class""},{""name"":""AsyncEvalsWithRawResponse"",""symbol_type"":""class""},{""name"":""EvalsWithStreamingResponse"",""symb...","{""symbols"":[{""name"":""Evals"",""symbol_type"":""class""},{""name"":""runs"",""symbol_type"":""property""},{""name"":""with_raw_response"",""symbol_type"":""property""},{""name"":""with_streaming_response"",""symbol_type"":""property""},{""name"":""create"",""symbol_type"":""function""},{..."


### 結果の可視化

以下は、構造化QA評価の評価データとコード出力を表す可視化です。これらの画像は、データ分布と評価ワークフローに関する洞察を提供します。

---

**評価データ概要**

![評価データ パート1](../../../images/eval_qa_data_1.png)

![評価データ パート2](../../../images/eval_qa_data_2.png)

---

**評価コードワークフロー**

![評価コード構造](../../../images/eval_qa_code.png)

---

これらの可視化を確認することで、評価データセットの構造とQAタスクの構造化出力を評価する際の手順をより良く理解できます。

### 使用例2: 多言語感情抽出
同様の方法で、構造化出力を持つ多言語感情抽出モデルを評価してみましょう。

In [29]:
# Sample in-memory dataset for sentiment extraction
sentiment_dataset = [
    {
        "text": "I love this product!",
        "channel": "twitter",
        "language": "en"
    },
    {
        "text": "This is the worst experience I've ever had.",
        "channel": "support_ticket",
        "language": "en"
    },
    {
        "text": "It's okay – not great but not bad either.",
        "channel": "app_review",
        "language": "en"
    },
    {
        "text": "No estoy seguro de lo que pienso sobre este producto.",
        "channel": "facebook",
        "language": "es"
    },
    {
        "text": "总体来说，我对这款产品很满意。",
        "channel": "wechat",
        "language": "zh"
    },
]

In [31]:
# Define output schema
sentiment_output_schema = {
    "type": "object",
    "properties": {
        "sentiment": {
            "type": "string",
            "description": "overall label: positive / negative / neutral"
        },
        "confidence": {
            "type": "number",
            "description": "confidence score 0-1"
        },
        "emotions": {
            "type": "array",
            "description": "list of dominant emotions (e.g. joy, anger)",
            "items": {"type": "string"}
        }
    },
    "required": ["sentiment", "confidence", "emotions"],
    "additionalProperties": False
}

# Grader prompts
sentiment_grader_system = """You are a strict grader for sentiment extraction.
Given the text and the model's JSON output, score correctness on a 1-5 scale."""

sentiment_grader_user = """Text: {{item.text}}
Model output:
{{sample.output_json}}
"""

In [32]:
# Register an eval for the richer sentiment task
sentiment_eval = client.evals.create(
    name="sentiment_extraction_eval",
    data_source_config={
        "type": "custom",
        "item_schema": {          # matches the new dataset fields
            "type": "object",
            "properties": {
                "text": {"type": "string"},
                "channel": {"type": "string"},
                "language": {"type": "string"},
            },
            "required": ["text"],
        },
        "include_sample_schema": True,
    },
    testing_criteria=[
        {
            "type": "score_model",
            "name": "Sentiment Grader",
            "model": "o3",
            "input": [
                {"role": "system", "content": sentiment_grader_system},
                {"role": "user",   "content": sentiment_grader_user},
            ],
            "range": [1, 5],
            "pass_threshold": 3.5,
        }
    ],
)

In [ ]:
# Run the sentiment eval
sentiment_run = client.evals.runs.create(
    name="gpt-4.1-sentiment",
    eval_id=sentiment_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in sentiment_dataset],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {"type": "input_text", "text": "You are a helpful assistant."},
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "{{item.text}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 100,
            "top_p": 0.9,
            "text": {
                "format": {
                    "type": "json_schema",
                    "name": "sentiment_output",
                    "schema": sentiment_output_schema,
                    "strict": True,
                },
            },
        },
    },
)

### 評価データの可視化
![image](../../../images/evals_sentiment.png)

### 概要と次のステップ

このノートブックでは、OpenAI Evaluation APIを使用して、構造化出力タスクにおけるモデルのパフォーマンスを評価する方法を実演しました。

**次のステップ：**
- 独自のモデルとデータセットでAPIを試してみることをお勧めします。
- APIの使用方法の詳細については、APIドキュメントも参照できます。

詳細については、[OpenAI Evalsドキュメント](https://platform.openai.com/docs/guides/evals)をご覧ください。